**Table of contents**<a id='toc0_'></a>    
1. [元类](#toc1_)    
1.1. [类也是对象](#toc1_1_)    
1.2. [动态地创建类](#toc1_2_)    
1.3. [使用type创建类](#toc1_3_)    
1.4. [使用type创建带有属性的类](#toc1_4_)    
1.5. [使用type创建带有方法的类](#toc1_5_)    
1.6. [到底什么是元类（终于到主题了）](#toc1_6_)    
1.7. [__metaclass__属性](#toc1_7_)    
1.8. [自定义元类](#toc1_8_)    
1.9. [究竟为什么要使用元类？](#toc1_9_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[元类](#toc0_)

## 1.1. <a id='toc1_1_'></a>[类也是对象](#toc0_)

在大多数编程语言中，类就是一组用来描述如何生成一个对象的代码段。在python中这一点仍然成立：

In [14]:
class ObjectCreator(object):
    pass


my_object = ObjectCreator()
print(my_object)

但是，Python中的类还远不止如此。类同样也是一种对象。是的，没错，就是对象。只要你使用关键字class，Python解释器在执行的时候就会创建一个对象。

下面的代码段：

In [15]:
class ObjectCreator(object):
    pass

将在内存中创建一个对象，名字就是ObjectCreator。这个对象（类对象ObjectCreator）拥有创建对象（实例对象）的能力。

但是，它的本质仍然是一个对象，于是乎你可以对它做如下的操作：

1. 你可以将它赋值给一个变量
2. 你可以拷贝它
3. 你可以为它增加属性
4. 你可以将它作为函数参数进行传递

下面是示例：

In [16]:
class ObjectCreator(object):
    pass


print(ObjectCreator)  # 你可以打印一个类，因为它其实也是一个对象


def echo(o):
    print(o)


echo(ObjectCreator)  # 你可以将类做为参数传给函数

print(hasattr(ObjectCreator, "new_attribute"))

# 你可以为类增加属性
ObjectCreator.new_attribute = "foo"  # type: ignore

print(ObjectCreator.new_attribute)  # type: ignore

ObjectCreatorMirror = ObjectCreator  # 你可以将类赋值给一个变量
print(ObjectCreatorMirror())

<class '__main__.ObjectCreator'>
<class '__main__.ObjectCreator'>
False
foo


## 1.2. <a id='toc1_2_'></a>[动态地创建类](#toc0_)
类也是对象，你可以在运行时创建它们，就像其他任何对象一样。

可以在函数中创建类，使用class关键字即可

In [17]:
def choose_class(name):
    if name == "foo":

        class Foo(object):
            pass

        return Foo  # 返回的是类，不是类的实例
    else:

        class Bar(object):
            pass

        return Bar


MyClass = choose_class("foo")
print(MyClass)  # 函数返回的是类，不是类的实例
print(MyClass())  # 你可以通过这个类创建类实例，也就是对象

<class '__main__.choose_class.<locals>.Foo'>
<__main__.choose_class.<locals>.Foo object at 0x00000228EA334220>


但这还不够动态，因为你仍然需要自己编写整个类的代码

由于类也是对象，所以它们肯定是通过什么东西来生成的才对

当使用class关键字时，Python解释器自动创建这个对象。但和Python中的大多数事情一样，Python仍然提供给你手动处理的方法

还记得内建函数type吗？这个古老但强大的函数能够让你知道一个对象的类型是什么，就像这样：

In [18]:
print(type(1))  # 数值的类型

print(type("1"))  # 字符串的类型

print(type(ObjectCreator()))  # 实例对象的类型

print(type(ObjectCreator))  # 类的类型

<class 'int'>
<class 'str'>
<class '__main__.ObjectCreator'>
<class 'type'>


仔细观察上面的运行结果，发现使用type对ObjectCreator查看类型是，答案为type， 是不是有些惊讶。。。看下面

## 1.3. <a id='toc1_3_'></a>[使用type创建类](#toc0_)
type还有一种完全不同的功能：创建类

type可以接受一个类的描述作为参数，然后返回一个类。

注意：

> type会根据传入的参数不同，执行不同的事情，例如type("hello")表示获取"hello"的类型，而传入的参数不同还能够表示创建一个类</br>
> 所以大家要注意：尽量不要让一个函数拥有2种完全不同的功能，这样会很傻，Python之所以将type设计为2个功能是为了保持向后兼容性

type可以像这样工作：

type(类名, 由父类名称组成的元组（针对无继承的情况，可以为空），包含属性的字典（名称和值）)

比如下面的代码：

In [19]:
# 按照普通定义类的方式
class A:
    pass


print(help(A))

# 使用type这种特殊方式
B = type("B", (), {})

print(help(B))

Help on class A in module __main__:

class A(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

None
Help on class B in module __main__:

class B(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

None


## 1.4. <a id='toc1_4_'></a>[使用type创建带有属性的类](#toc0_)
type 接受一个字典来为类定义属性，因此

In [20]:
Foo = type("Foo", (), {"bar": True})

可以翻译为

In [21]:
class Foo:
    bar = True

并且可以将Foo当成一个普通的类一样使用：

In [22]:
Foo2 = type("Foo2", (), {"bar": True})

print(Foo2)
print(Foo2.bar)
f = Foo2()
print(f)
print(f.bar)

<class '__main__.Foo2'>
True
True


当然，你可以继承这个类，代码如下：

In [23]:
class Foo2Children(Foo2):
    pass

就可以写成：

In [24]:
Foo2Children = type("Foo2Children", (Foo2,), {})
print(Foo2Children)

print(Foo2Children.bar) # # bar属性是由Foo继承而来

<class '__main__.Foo2Children'>
True


**注意：**
>- type的第2个参数，元组中是父类的名字，而不是字符串
>- 添加的属性是类属性，并不是实例属性

## 1.5. <a id='toc1_5_'></a>[使用type创建带有方法的类](#toc0_)
最终你会希望为你的类增加方法。只需要定义一个有着恰当签名的函数并将其作为属性赋值就可以了。

**添加实例方法**

In [13]:
# 定义了一个普通函数
def echo_bar(self):
    print(self.bar)


FooChildren = type("FooChildren", (Foo,), {"echo_bar": echo_bar})

print(hasattr(Foo, "echo_bar"))  # 判断Foo类中 是否有echo_bar这个属性

print(hasattr(FooChildren, "echo_bar"))  # 判断FooChild类中 是否有echo_bar这个属性

my_foo = FooChildren()
my_foo.echo_bar()

False
True
True


**添加静态方法**

In [26]:
@staticmethod
def test_static():
    print('staticmethod...')
    
Foochild = type('Foochild', (Foo,), {"echo_bar": echo_bar, "test_static": test_static})

fooclid = Foochild()

print(fooclid.test_static)

fooclid.test_static()

fooclid.echo_bar()


<function test_static at 0x00000228E8FA3DC0>
staticmethod...
True


**添加类方法**

In [27]:
@classmethod
def test_class(cls):
    print(cls.bar)

Foochild = type('Foochild', (Foo,), {"echo_bar":echo_bar, "test_static": test_static, "test_class": test_class})

fooclid = Foochild()

fooclid.test_class()

True


你可以看到，在Python中，类也是对象，你可以动态的创建类。这就是当你使用关键字class时Python在幕后做的事情，而这就是通过元类来实现的。

较为完整的使用type创建类的方式:

In [29]:
class A(object):
    num = 100

def print_b(self):
    print(self.num)


@staticmethod
def print_static():
    print("----haha-----")


@classmethod
def print_class(cls):
    print(cls.num)


# 下面通过type来创建一个类，
# 名字是B
# 继承A
# 有3个属性，因为它们指向函数（第5、10，15行定义的函数），所以也称之为定义了3个方法
B = type("B", (A,), {"print_b": print_b, "print_static": print_static, "print_class": print_class})
b = B()  # 创建实例对象
b.print_b()  # 调用实例方法
b.print_static()  # 调用静态方法
b.print_class()  # 调用类方法


100
----haha-----
100


## 1.6. <a id='toc1_6_'></a>[到底什么是元类（终于到主题了）](#toc0_)
元类就是用来创建类的“东西”。你创建类就是为了创建类的实例对象，不是吗？但是我们已经学习到了Python中的类也是对象。

元类就是用来创建这些类（对象）的，元类就是类的类，你可以这样理解为：

>MyClass = MetaClass()  # 使用元类创建出一个对象，这个对象称为“类”

>my_object = MyClass()  # 使用“类”来创建出实例对象

你已经看到了type可以让你像这样做：
>MyClass = type('MyClass', (), {})

这是因为函数type实际上是一个元类。type就是Python在背后用来创建所有类的元类。</br>
现在你想知道那为什么type会全部采用小写形式而不是Type呢？</br>
好吧，我猜这是为了和str保持一致性，str是用来创建字符串对象的类，而int是用来创建整数对象的类。</br>
type就是创建类对象的类。</br>
你可以通过检查__class__属性来看到这一点。Python中所有的东西，注意，我是指所有的东西——都是对象。</br>
这包括整数、字符串、函数以及类。它们全部都是对象，而且它们都是从一个类创建而来，这个类就是type。</br>

In [31]:
# 测试数字的类
age = 35
print(age.__class__)


# 测试字符串的类
name = 'bob'
print(name.__class__)


# 测试函数的类
def Foo():
    pass

print(Foo.__class__)


# 测试实例对象的的类
class Bar(object):
    pass

b = Bar()
print(b.__class__)


# 测试类的类
print(Bar.__class__)


<class 'int'>
<class 'str'>
<class 'function'>
<class '__main__.Bar'>
<class 'type'>


现在，对于任何一个__class__的__class__属性又是什么呢？

In [32]:
# 测试数字的类
age = 35
print(age.__class__)


# 测试字符串的类
name = 'bob'
print(name.__class__)


# 测试函数的类
def Foo():
    pass

print(Foo.__class__)


# 测试实例对象的的类
b = Bar()
print(b.__class__)


# 测试类的类
class Bar(object):
    pass


print(Bar.__class__)

print("------分割线-------")

# 测试类型的类
print(age.__class__.__class__)
print(name.__class__.__class__)
print(Foo.__class__.__class__)
print(b.__class__.__class__)
print(Bar.__class__.__class__)  # type类的元类是type。。。。


<class 'int'>
<class 'str'>
<class 'function'>
<class '__main__.Bar'>
<class 'type'>
------分割线-------
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


因此，元类就是创建类这种对象的东西。type就是Python的内建元类，当然了，你也可以创建自己的元类。

## 1.7. <a id='toc1_7_'></a>[__metaclass__属性](#toc0_)
你可以在定义一个类的时候为其添加__metaclass__属性

    class Foo(object):
        __metaclass__ = something…
        ...省略...

如果你这么做了，Python就会用元类来创建类Foo。小心点，这里面有些技巧。</br>
你首先写下class Foo(object)，但是类Foo还没有在内存中创建。</br>
Python会在类的定义中寻找__metaclass__属性，</br>
如果找到了，Python就会用它来创建类Foo，如果没有找到，就会用内建的type来创建这个类。</br>

把下面这段话反复读几次

当你写如下代码时 :

In [ ]:
class Foo(Bar):
    pass

Python做了如下的操作：

1. Foo中有__metaclass__这个属性吗？如果有，则Python会通过__metaclass__创建一个名字为Foo的类
2. 如果Python没有找到__metaclass__，它会继续在Bar（父类）中寻找__metaclass__属性，并尝试做和前面同样的操作
3. 如果Python在任何父类中都找不到__metaclass__，它就会在模块层次中去寻找__metaclass__，并尝试做同样的操作
4. 如果还是找不到__metaclass__,Python就会用内置的type来创建这个类对象

现在的问题就是，可以在__metaclass__中放置些什么代码呢？

答：可以创建一个类的东西

那么什么可以用来创建一个类呢？

答：type，或者任何使用到type或者子类化type的东东都可以

## 1.8. <a id='toc1_8_'></a>[自定义元类](#toc0_)
元类的主要目的就是为了当创建类时能够自动地改变类。

假想一个很傻的例子，你决定在你的模块里所有的类的属性都应该是大写形式。</br>
有好几种方法可以办到，但其中一种就是通过在模块级别设定__metaclass__采用这种方法，</br>
这个模块中的所有类都会通过这个元类来创建，我们只需要告诉元类把所有的属性都改成大写形式就万事大吉了。</br>

幸运的是，__metaclass__实际上可以被任意调用，它并不需要是一个正式的类

所以，我们这里就先以一个简单的函数作为例子开始，看如下代码：

Python3中

In [35]:
def upper_attr(class_name, class_parents, class_attr):
    # class_name 会保存类的名字 Foo
    # class_parents 会保存类的父类 object
    # class_attr 会以字典的方式保存所有的类属性

    # 遍历属性字典，把不是__开头的属性名字变为大写
    new_attr = {
        name.upper(): value
        for name, value in class_attr.items()
        if not name.startswith("__")
    }
    # 调用type来创建一个类
    return type(class_name, class_parents, new_attr)

class Foo(object,metaclass=upper_attr):
    # * python2 中的用法
    # __metaclass__ = upper_attr  # 设置Foo类的元类为upper_attr
    bar = 'bip'

print(hasattr(Foo, 'bar'))
print(hasattr(Foo, 'BAR'))

f = Foo()
print(f.BAR)



False
True
bip


现在让我们用一个真正的class来当做元类

In [5]:
class UpperAttrMetaClass(type):
    # __new__ 是在__init__之前被调用的特殊方法
    # __new__是用来创建对象并返回之的方法
    # 而__init__只是用来将传入的参数初始化给对象
    # 你很少用到__new__，除非你希望能够控制对象的创建
    # 这里，创建的对象是类，我们希望能够自定义它，所以我们这里改写__new__
    # 如果你希望的话，你也可以在__init__中做些事情
    # 还有一些高级的用法会涉及到改写__call__特殊方法，但是我们这里不用
    def __new__(cls, class_name, class_parents, class_attr):
        print("===>UpperAttrMetaClass.__new__")
        print(cls)
        print(class_name)
        print(class_parents)
        print(class_attr)
        
        # 遍历属性字典，把不是__开头的属性名字变为大写
        new_attr = {
            name.upper(): value
            for name, value in class_attr.items()
            if not name.startswith("__")
        }
        # 方法1：通过'type'来做类对象的创建
        return type(class_name, class_parents, new_attr)

        # 方法2：复用type.__new__方法
        # 这就是基本的OOP编程，没什么魔法
        # return type.__new__(cls, class_name, class_parents, new_attr)

# python3的用法
class Foo(object, metaclass=UpperAttrMetaClass):
    bar = 'bip'

# python2的用法
# class Foo(object):
#     __metaclass__ = UpperAttrMetaClass
#     bar = 'bip'


print(hasattr(Foo, 'bar'))
# 输出: False
print(hasattr(Foo, 'BAR'))
# 输出:True

f = Foo()
print(f.BAR)
# 输出:'bip'


Writing test_metaclass.py


就是这样，除此之外，关于元类真的没有别的可说的了

但就元类本身而言，它们其实是很简单的：

1. 拦截类的创建
2. 修改类
3. 返回修改之后的类

## 1.9. <a id='toc1_9_'></a>[究竟为什么要使用元类？](#toc0_)
现在回到我们的大主题上来，究竟是为什么你会去使用这样一种容易出错且晦涩的特性？好吧，一般来说，你根本就用不上它：

“元类就是深度的魔法，99%的用户应该根本不必为此操心。</br>
如果你想搞清楚究竟是否需要用到元类，那么你就不需要它。</br>
那些实际用到元类的人都非常清楚地知道他们需要做什么，而且根本不需要解释为什么要用元类。” —— Python界的领袖 Tim Peters

当然了也不是说元类真的没有用，我们在后面学习MySQL数据库后，会学习使用元类来自己模拟ORM，到时就要用到元类